In [1]:
#
# cs590-Module7-neo4j.ipynb - This is a Jupyter Notebook designed to 
# connect to a Neo4j database and provide CRUD operations.
#
# This notebook uses a supplemental configuration file that has the
# following format:
# 
# [neo4j]
# host=localhost
# database=maxiyum
# user=neo4j
# password=REPLACE_WITH_YOUR_PASSWORD_FOR_THE_DATABASE_USER
#
# We also build a config function to read the data from the
# configuration file to make the code easier to maintain and more
# secure. 
#
# Remember: You should _NEVER_ embed a password in your 
# source code!
#
#------------------------------------------------------------------
# Change History
#------------------------------------------------------------------
# Version   |   Description
#------------------------------------------------------------------
#    1          Initial Development
#------------------------------------------------------------------

from neo4j import GraphDatabase
from configparser import ConfigParser

#
# db - variable to identify the database we are communicating with
#
db = 'maxiyum'

#
# config - This function is designed to read the data from our
# configuration file to allow us to setup the connection to our
# Neo4j database.
#
# Please Note: We create this method genericly so that it can
# be reused with other configuration data separated by section
# as key-value pairs
#
# Parameters:
#
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
# 
# Returns:
#   A dictionary with the key-value pairs from the configuration file.
#
def config(filename, section):
    # Initialize a configuration parser
    cp = ConfigParser()
    
    # Acquire data from the configuration file
    cp.read(filename)
    
    # Import configuration section as a dictionary
    block = {}
    
    # Check to make sure that the section exists in our config file
    if cp.has_section(section):
        # Read the entire section
        items = cp.items(section)
        #print(items)
        
        # Loop through config items to create the dictionary
        for item in items:
            block[item[0]] = item[1]
    else:
        # If we have gotten here the section did not exist in the
        # config file, so we raise an exception
        raise Exception(f"Section {section} was not found in the {filename} file!")
    
    # Return the dictionary to the calling program
    return block


#
# displayConfig - This function is designed to test the config function.
# This function will call the config function with a filename and a section
# obtain the resulting data, and then print the retrieved information to
# the screen. This can be useful for debugging and testing purposes.
#
# Parameters:
#
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
# 
def displayConfig(filename, section):
    # Obtain configuration data from the config file
    confData = config(filename, section)
    
    # Display results
    print(confData)
    

#
# dbConnect - function designed to connect to the database. This function
# leverages the config method to pull the necessary data from our database
# configuration file.
#
# Parameters:
#
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
# 
# Returns a Connection to the databse or null in the event of a problem
#
def dbConnect(filename, section):
    # Obtain configuration data from the config file
    confData = config(filename, section)

    # Create a try/except block for the connection. This is important
    # in order to keep from blowing up the application in the event that
    # we cannot connect to the database.
    try:     
        # Setup the connection to the MongoDB server
        URI = "neo4j://" + confData["host"] + "/" + confData["database"]
        AUTH = (confData["user"],confData["password"])
        
        conn = GraphDatabase.driver(URI,auth=AUTH)
        conn.verify_connectivity()
        # Return the database connection for use in the application
        # Don't forget to close the connection when done.
        return conn
    except(Exception) as error:
        # We had a problem, so lets see what it was
        print(error)
        return None
    
#
# dbTest - a utility method to test connection to the database
# This method will connect to the database with the information from
# the configuration file, validate that the connection was made to the 
# Appropriate database, print the result, and then close the database
# connection. This function is useful for debugging initial connections. 
# 
# Parameters:
#
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
#
def dbTest(filename, section):
    # Create a try/except block for the connection. This is important
    # in order to keep from blowing up the application in the event that
    # we cannot connect to the database.
    try:     
        # create our connection
        print('Connecting to our Neo4j database...')
        conn = dbConnect(filename, section)
    
        # How many documents are in our collection?
        conn.verify_connectivity()
        print("Connection Verified!")
    except(Exception) as error:
        # We had a problem, so lets see what it was
        print(error)
        
    # Everything worked well, so we need to close the connection
    if conn is not None:
        conn.close()


#
# create - CRUD Method used to create a new node in the graph.
# This method will be used to create a new node in the graph
# database. Because the graph database can have many different
# types of nodes, this method is very generic. You will have
# to pass the necessary cypher commands to add the node.
#
# This method will use the gloal variable db to indicate the
# database to execute the query against.
#
# Parameters:
#     cypher: The necessary cypher code to add the node
#     conn: The database connection to use create the record
#
def createRecord(cypher, conn):
    results = []
    with conn.session(database=db) as session:
        res = session.run(cypher)
        results = list(res)
    return results
                                      

#
# testCreateRecord - Method used to test the createRecord function.
# This function will exercise the createRecord function by attempting to add
# a Nutrient Description (NutDes) record to the graph. This function will
# only work with the Nutrient database, it is not portable.
#
# Parameters:
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
#
def testCreateRecord(filename, section):
    # create our connection
    print('Connecting to our Neo4j database...')
    conn = dbConnect(filename, section)

    # Setup the cypher code

    cypher = 'MERGE (n:NutDes { NutrientCode: 777, NutrientDescription: "Zinc", NutrientDescriptionAbbrev: "Zn", NutrientUnit: "mcg", DateAdded: "2025-05-08", LastModified: "2025-05-10"})'
             
    # cypher = "match (n:NutDes) return count(n)"

    # Add record to the collection
    res = createRecord(cypher,conn)
    for i in res:
        print(i)
    print('Record Added')

    # Cleanup and close connection
    conn.close()


#
# readRecord - CRUD Method used to read a record from a database. 
# This method will be used to read a node from the graph.
# Because Neo4j is a graph database, this operation is specific
# to graph nodes and edges
#
# Parameters:
#     query: cypher query used to return one or more records
#     conn: The database connection to use create the record
#    
def readRecord(query, conn):
    with conn.session(database=db) as session:
        res = session.run(query)
        records = list(res)
        return records if records else None


#
# testReadRecord - Method used to test the readRecord function.
# This function will exercise the readRecord function by attempting to read
# a specific NutDes (Nutrient Description) node. This function will
# only work with the Nutrient database, it is not portable.
#
# Parameters:
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
#
def testReadRecord(filename, section):
    # create our connection
    print('Connecting to our Neo4j database...')
    conn = dbConnect(filename, section)
    
    # Setup dictionary to find data
    query = 'MATCH (n:NutDes) where n.NutrientCode = 777 return n'

    # Find records if they exist
    records = readRecord(query, conn)
    
    # Display results
    print('Records Retrieved:')
    if records is not None:
        for i in records:
            print(i)
    else:
        print('There were no records retrieved.')

    # Cleanup and close connection
    conn.close()

#
# updateRecord - CRUD Method used to update a record from a database. 
# This method will be used to update a node or edge in the graph.
# Because Neo4j is a graph database, this will update a operate on
# a nodes or edges.
#
# Parameters:
#     query: cypher statement used to update record(s)
#     conn: The database connection to use create the record
#    
def updateRecord(query, conn):
    with conn.session(database=db) as session:
        res = session.run(query)
        return list(res)
    
#
# testUpdateRecord - Method used to test the updateRecord function.
# This function will exercise the updateRecord function by attempting to update
# a Nutrient Description record. This function will
# only work with the Nutrient database, it is not portable.
#
# Parameters:
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
#
def testUpdateRecord(filename, section):
    # create our connection
    print('Connecting to our Neo4j database...')
    conn = dbConnect(filename, section)

    # Configure cypher query
    query = 'MATCH (n:NutDes {NutrientCode: 777}) SET n.NutrientDescription = "ZINC" RETURN n'
    
    # Process test
    records = updateRecord(query, conn)   

    # Print results
    print(f'{len(records)} record(s) have been updated.')
    conn.close()


#
# deleteRecord - CRUD Method used to delete a record from a database. 
# This method will be used to delete a node or edge from the graph.
# Because Neo4j is a document database, this will remove a record from a
# graph.
#
# NOTE: USE WITH CAUTION! THIS METHOD WILL DESTROY DATA
#
# Parameters:
#     query: The cypher code used to select the nodes to delete
#     conn: The database connection to use create the record
#    
def deleteRecord(query, conn):
    with conn.session(database=db) as session:
        res = session.run(query)
        return list(res)

#
# testDeleteRecord - Method used to test the deleteRecord function.
# This function will exercise the deleteRecord function by attempting to remove
# a record from the graph. This function will
# only work with the Nutrient database, it is not portable.
#
# Parameters:
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
#
def testDeleteRecord(filename, section):
    # create our connection
    print('Connecting to our Neo4j database...')
    conn = dbConnect(filename, section)

    # setup the dictionary to select the record(s) to remove
    query = 'MATCH (n:NutDes {NutrientCode: 777}) DELETE n RETURN COUNT(*) AS deleted'

    
    # execute record removal
    records = deleteRecord(query, conn)    

    # Report results
    # We would report results here, but there are no records returned from a delete operation.
    print(f'Delete Result: {records}')
        
    conn.close()

if __name__ == '__main__':
    displayConfig('database.conf','neo4j')
    dbTest('database.conf','neo4j')
    testCreateRecord('database.conf','neo4j')
    testReadRecord('database.conf','neo4j')
    testUpdateRecord('database.conf','neo4j')
    testDeleteRecord('database.conf','neo4j')

{'host': 'localhost', 'port': '7687', 'database': 'neo4j', 'user': 'neo4j', 'password': 'cs590password'}
Connecting to our Neo4j database...
'NoneType' object has no attribute 'on_neo4j_error'
'NoneType' object has no attribute 'verify_connectivity'
Connecting to our Neo4j database...
'NoneType' object has no attribute 'on_neo4j_error'


AttributeError: 'NoneType' object has no attribute 'session'